In [1]:
import geopandas as gpd

blocks = gpd.read_parquet('./../data/blocks.parquet')
columns = [c for c in blocks.columns if 'capacity_' in c]
blocks = blocks[['geometry', 'population', 'site_area', *columns]].copy()
blocks.head(3)

,geometry,population,site_area,capacity_convenience,capacity_cafe,capacity_mall,capacity_pharmacy,capacity_bank,capacity_fuel,capacity_pitch,...,capacity_bus_station,capacity_bus_stop,capacity_pier,capacity_animal_shelter,capacity_military_kom,capacity_prison,capacity_landfill,capacity_plant_nursery,capacity_greenhouse_complex,capacity_warehouse
id,,,,,,,,,,,,,,,,,,,,,
0,"POLYGON ((354918.622 6625258.829, 354901.464 6...",606,804466.712114,208.0,406.0,2216.0,474.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,"POLYGON ((355412.142 6623378.149, 355411.700 6...",41,23173.129862,811.0,0.0,0.0,0.0,148.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,"POLYGON ((353934.329 6625429.433, 353923.453 6...",14,363005.815802,0.0,0.0,0.0,0.0,0.0,501.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
import pandas as pd

accessibility_matrix = pd.read_pickle('./../data/accessibility_matrix.pickle')
accessibility_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,16310,16311,16312,16313,16314,16315,16316,16317,16318,16319
0,0.000000,18.718750,9.234375,17.21875,30.296875,27.390625,30.671875,23.281250,78.6875,81.2500,...,76.5625,102.5625,97.5000,132.250,130.8750,142.125,247.625,228.625,247.625,124.6250
1,17.859375,0.000000,19.015625,7.65625,20.281250,25.250000,25.234375,15.765625,76.8750,79.4375,...,74.6875,100.7500,95.6875,130.500,129.0000,140.250,245.875,226.875,245.875,112.5000
2,9.234375,20.796875,0.000000,20.21875,27.968750,25.062500,28.343750,20.953125,71.8750,74.4375,...,69.7500,95.8125,90.6875,125.500,124.0625,135.375,240.875,221.875,240.875,124.6250
3,15.937500,7.449219,18.421875,0.00000,21.109375,24.984375,27.781250,16.593750,76.3125,78.8125,...,74.1250,100.1875,95.0625,129.875,128.5000,139.750,245.250,226.250,245.250,113.3750
4,31.906250,19.765625,31.734375,21.28125,0.000000,21.781250,23.125000,15.429688,89.5625,92.1250,...,87.4375,113.5000,108.3750,143.125,141.7500,153.000,258.500,239.500,258.500,108.3125


## Simulated annealing

In [3]:
from blocksnet.enums import LandUse

blocks_lus = {
    # 1 : LandUse.INDUSTRIAL,
    # 2 : LandUse.RECREATION,
    1536 : LandUse.RESIDENTIAL
}

service_types = {
    "kindergarten": 0.2,
    "school": 0.2,
    "pharmacy": 0.2,
    "polyclinic": 0.3,
    "convenience": 0.2,   
    "cafe": 0.15,
    "playground": 0.2,
    "post": 0.1,
    "hairdresser": 0.1
}

In [ ]:
from blocksnet.optimization.services import ServicesOptimizer

so = ServicesOptimizer(blocks[['site_area', 'population']], accessibility_matrix)
for service_type, weight in service_types.items():
    so.add_service_type(service_type, weight, blocks.rename(columns={f'capacity_{service_type}': 'capacity'})[['capacity']])

so.service_types

Index(['geometry', 'population', 'site_area', 'capacity_convenience',
       'capacity_cafe', 'capacity_mall', 'capacity_pharmacy', 'capacity_bank',
       'capacity_fuel', 'capacity_pitch', 'capacity_playground',
       'capacity_museum', 'capacity_parking', 'capacity_bakery',
       'capacity_restaurant', 'capacity_dog_park', 'capacity_university',
       'capacity_police', 'capacity_bar', 'capacity_park',
       'capacity_government', 'capacity_supermarket', 'capacity_kindergarten',
       'capacity_polyclinic', 'capacity_school', 'capacity_religion',
       'capacity_post', 'capacity_hairdresser', 'capacity_theatre',
       'capacity_stadium', 'capacity_cemetery', 'capacity_substation',
       'capacity_hotel', 'capacity_hospital',
       'capacity_multifunctional_center', 'capacity_swimming_pool',
       'capacity_beach', 'capacity_train_building',
       'capacity_wastewater_plant', 'capacity_hostel', 'capacity_cinema',
       'capacity_water_works', 'capacity_market', 'capacity_

{'kindergarten': 0.2,
 'school': 0.2,
 'pharmacy': 0.2,
 'polyclinic': 0.3,
 'convenience': 0.2,
 'cafe': 0.15,
 'playground': 0.2,
 'post': 0.1,
 'hairdresser': 0.1}

In [ ]:
variables_df, objective_value = so.run(blocks_lus, iterations=1000)

2025-04-06 01:15:13.518 | INFO     | blocksnet.optimization.services.old_core:_initialize_provisions_dfs:52 - Initial provision assessment
  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
variables_df

,block_id,service_type,capacity,site_area,build_floor_area,count,total_capacity,total_site_area,total_build_floor_area
0,1536,school,250,18500.0,2500.0,0,0,0.0,0.0
1,1536,school,300,15000.0,2400.0,0,0,0.0,0.0
2,1536,school,600,30000.0,4800.0,1,600,30000.0,4800.0
3,1536,school,1100,33000.0,6600.0,2,2200,66000.0,13200.0
4,1536,school,250,200.0,2200.0,0,0,0.0,0.0
5,1536,school,300,300.0,3600.0,1,300,300.0,3600.0
6,1536,school,600,600.0,7100.0,0,0,0.0,0.0
7,1536,kindergarten,80,3520.0,240.0,2,160,7040.0,480.0
8,1536,kindergarten,180,6840.0,540.0,1,180,6840.0,540.0
9,1536,kindergarten,320,12160.0,960.0,1,320,12160.0,960.0
